<a href="https://colab.research.google.com/github/jwalor/mineriaweb/blob/main/Analisis_Sentimiento_tf_idf_svm_PUCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de Sentimientos (tf-idf + SVM)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

## 1. Cargando Datos

In [6]:
pd.ExcelFile("/content/drive/My Drive/mineria web/s6/80s.xlsx")

In [8]:
archivo = pd.ExcelFile("/content/drive/My Drive/mineria web/s6/80s.xlsx")
archivo.sheet_names

['80s']

In [ ]:
df = archivo.parse("80s")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['Comentariomin'].values[0]

#### Separando comentarios positivos/negativos

In [ ]:
df_positivos = df[df["EVALUACION"]>=4]
df_negativos = df[df["EVALUACION"]<=2]

In [ ]:
df_positivos.shape, df_negativos.shape

In [ ]:
datos = pd.concat([df_positivos,df_negativos],ignore_index=True)
datos.shape

In [ ]:
datos['EVALUACION'][datos["EVALUACION"]<=2] = 0
datos['EVALUACION'][datos["EVALUACION"]>=4] = 1

In [ ]:
datos

## 2. Limpieza básica de Datos Textuales

In [ ]:
def limpieza(texto):
    
    out = []
    for w in texto.split():
        if (len(w)>3):
            out.append(w)
    
    return " ".join(out)

In [ ]:
datos['Limpio'] = datos['Comentariomin'].apply(limpieza)
datos.head()

## 3. TF - TF-idf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
data = datos['Limpio'].values.tolist()

In [ ]:
vectorizer = TfidfVectorizer(max_features=100)
FV = vectorizer.fit_transform(data)
FV = FV.toarray()

In [ ]:
vectorizer.get_feature_names()

In [ ]:
datos.head()

In [ ]:
datos['FV'] = FV.tolist()
datos.head()

#### Separando Train/Test (80%/20%)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = np.array(datos.FV.to_list())
Y = np.array(datos.EVALUACION.to_list())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    Y,
                                                    test_size=0.2,
                                                   random_state=10)

In [ ]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

## 4. SVM

In [ ]:
from sklearn import svm

In [ ]:
clasificador = svm.SVC()
clasificador.fit(X_train,y_train)

In [ ]:
Y_pred = clasificador.predict(X_test)

In [ ]:
Y_pred

In [ ]:
from sklearn.metrics import accuracy_score,classification_report

In [ ]:
accuracy_score(y_test,Y_pred)

In [ ]:
print(classification_report(y_test,Y_pred))

In [ ]:
texto = "rapida"
fv = vectorizer.transform([texto]).toarray()
clasificador.predict(fv)

In [ ]:
texto = "demora"
fv = vectorizer.transform([texto]).toarray()
clasificador.predict(fv)